<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/proc_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import math
import scipy.stats as sc

from bs4 import BeautifulSoup as bs
from bs4 import UnicodeDammit

In [248]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [249]:
df_ed = pd.read_csv('/content/drive/My Drive/dfs_tot (1).csv')
df_ed.head(n=20)

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN,Documento
0,1808: Como uma rainha,Laurentino Gomes,384,Globo Livros,(Português),Capa Comum,978-8525057518,/content/drive/My Drive/9periodo/Recuperação...
1,Box - As Melhores Aventuras Do Sítio Do Picapa...,Monteiro Lobato,640,Nova Fronteira,(Português),Capa dura,978-8520943519,/content/drive/My Drive/9periodo/Recuperação...
2,Cotoco. O Diário De Um Garoto De 13 Anos,John Van de Ruit,390,Intrínseca,(Português),Capa Comum,978-8598078854,/content/drive/My Drive/9periodo/Recuperação...
3,"Doctor Who: 12 Doutores, 12 histórias",Alex Scarrow,480,Fantástica Rocco,(Português),Capa Comum,978-8568263044,/content/drive/My Drive/9periodo/Recuperação...
4,Freud (1900) A interpretação dos sonhos: Obras...,Sigmund Freud,736,Companhia das Letras,(Português),Capa dura,978-8535932218,/content/drive/My Drive/9periodo/Recuperação...
5,Good Omens: Belas Maldições,Neil Gaiman,364,Bertrand Brasil,(Português),Capa Comum,978-8528624021,/content/drive/My Drive/9periodo/Recuperação...
6,Os Mitos de Cthulhu - Volume Único (Exclusivo ...,Esteban Maroto,92,Pipoca e Nanquim (7 de junho de 2019),(Português),Capa dura,978-8593695261,/content/drive/My Drive/9periodo/Recuperação...
7,Redes de Computadores e a Internet: Uma Aborda...,Jim Kurose,656,Pearson Universidades,(Português),Capa Comum,978-8581436777,/content/drive/My Drive/9periodo/Recuperação...
8,Sobotta - Atlas de Anatomia Humana - 3 Volumes,Friedrich Paulsen,1168,Guanabara Koogan,(Português),Capa dura,978-8527732376,/content/drive/My Drive/9periodo/Recuperação...
9,Alice No País Das Maravilhas (Classic Edition),"Carroll,Lewis",208,NaN,Português,Capa dura,9788594541758,/content/drive/My Drive/9periodo/Recuperação...


# Funções

## Parse HTML

In [0]:
symbolsToReplace = [
    '(', ')', '!', '?', ':', ';', '*', '.', ',', '★', '|', '+', '[', ']', '{', '}', '/', 'ª', 'º', '°', '-',
    '%', '—', '@', '#',  '\"','\'', '<', '>', '=', '´', '`', '“', '$', '&', '’', '¡', '€', 'µ', '¦',
    '\\', '®', '™', '”', '…', '‘', '•', '😍','😙', '❤', '–', '⭐️', '_', '️⭐', '😉', '👏', '¹', '²', '³',
    "£", '¢', '¬', '§', '~', '^', '×', '÷',
    '\u200c', '\u200f', '\u200e', '\t', '\xa0', '\x03', '\u0301', '\u0303'
]

allowedSmallTokens = [
  'fio', 'box', 'god', 'key', 'bit', 'led', 'faq', 'dog', 'sim', 'ovo', 'pão', 'aba', 'new' ## acabei de perceber que 'não' nem tá aqui kkkkk
]

In [0]:
def getAndCleanHTML(filepath):
  
  page = open(filepath, "rb")

  page = page.read()

  clearPage = UnicodeDammit.detwingle(page)
  
  doc = bs(clearPage, 'lxml')
  
  for script in doc(["script", "style"]):
    script.extract()
  
  docText = doc.get_text(' ')
  
  docText = docText.translate({ord(ch): None for ch in '0123456789'})
  
  docText = docText.lower()
  
  docText = docText.replace("e-book", "ebook")

  docText = docText.replace("blu-ray", "bluray")
  
  for symbol in symbolsToReplace:
    docText = docText.replace(symbol, ' ')
  
  docText = docText.replace("  ", ' ')
  
  lines = (line.strip() for line in docText.splitlines())
  
  chunks = (phrase.strip() for line in lines for phrase in line.split(' '))

  text = ' '.join(chunk for chunk in chunks if chunk)
  
  return text

## Searching Index

In [0]:
def handlingQuery(query):
  hq = query.split()
  return hq

In [0]:
def getValues(query):
  rl = list()
  for i in query:
    wl = query[i].split(' ')
    for w in wl:
      rl.append(w)
  return rl

In [0]:
def free_search(q, indice):
  hq = handlingQuery(q)
  df_search = pd.DataFrame()
  for q in hq:
    for w in indice['Terms'].values:
      if (w in q) or (w == q):
        df_search = df_search.append(indice[indice['Terms'] == w], ignore_index=True)
  return df_search

In [0]:
def structured_search(sq, indice):
  df_search = pd.DataFrame()
  for i in sq:
    values_sq = sq[i].split(' ')
    for v in values_sq:
      for r in range(len(df_campos)):
        sr = df_campos.at[r, 'Field Text'].split('.')
        if (sr[0] in v) and (sr[1] == i):
          df_search = df_search.append(df_campos[df_campos['Field Text'] == sr[0] + '.' + sr[1]], ignore_index=True)
  return df_search

In [0]:
def docs_list_OR(df):
  postings = list()
  for r in df.index:
    postings.append(df.at[r,'Posting'])

  for i in range(len(postings)):
    new_postings = postings[i].split(', ')
    new_postings[0] = new_postings[0][1:]
    new_postings[len(new_postings)-1] = new_postings[len(new_postings)-1][:-1]
    postings[i] = new_postings

  p = list()
  for i in range(len(postings)):
    for j in range(len(postings[i])):
      p.append(postings[i][j])

  p = list(dict.fromkeys(p))
  return p

In [0]:
def docs_list_AND(df):
  aux = list()
  postings_list = list()
  #colocar postings em listas
  for r in df.index:
    aux = df.at[r, 'Posting'].split(', ')
    aux[0] = aux[0][1:]
    aux[len(aux)-1] = aux[len(aux)-1][:-1]
    postings_list.append(aux)

  #iterar sobre listas de postings buscando itens iguais
  postings = list()
  for i in range(len(postings_list)):
    for j in range(i+1, len(postings_list)):
      for k in range(len(postings_list[i])):
        for l in range(len(postings_list[j])):
          if postings_list[i][k] == postings_list[j][l]:
            postings.append(postings_list[i][k])
            postings_list[i][j] = ' '

  return postings

## Calc Scores

In [0]:
def calc_tfidf(df_freq):
  if(df_freq.index.size > 0):
    for r in df_freq.index:
      df_freq.at[r, 'Freq'] = math.log2(df_arquivos.shape[0]/df_freq.at[r, 'Freq'])
      for c in range(2, len(df_freq.columns)):
        df_freq.at[r, df_freq.columns[c]] = np.multiply(df_freq.at[r, 'Freq'], df_freq.at[r, df_freq.columns[c]])
  else:
    df_freq = pd.DataFrame()

  return df_freq

In [0]:
def calc_freq(dl, df_search, tipo_busca, queryl):
  if(df_search.index.size > 0):

    df_freq = pd.DataFrame(columns=dl)

    queryt = list()
    if tipo_busca == 'livre':
      df_freq = pd.concat([df_search['Terms'], df_search['Freq'], df_freq], axis=1)
      for q in queryl:
        for v in df_freq['Terms'].values:
          if v in q:
            queryt.append(q)
    else:
      df_freq = pd.concat([df_search['Field Text'], df_search['Freq'], df_freq], axis=1)
      for q in queryl:
        for v in df_freq['Field Text'].values:
          if v in q:
            queryt.append(q)

    for r in df_freq.index:
      for c in range(2, len(df_freq.columns)):
        df_freq.at[r, df_freq.columns[c]] = 0

    df_freq.iloc[:, 1:] = df_freq.iloc[:, 1:].astype(float)
    
    for i in range(len(dl)):
      doc = getAndCleanHTML(df_arquivos.loc[int(dl[i]),'0'])
      doc = doc.lower()
      doc = doc.split()
      for w in doc:
        for qw in range(len(queryt)):
          if w == queryt[qw]:
            df_freq.at[qw, dl[i]] += 1.0

    for r in df_freq.index:
      for c in range(2, len(df_freq.columns)):
        if df_freq.at[r,df_freq.columns[c]] !=0:
          df_freq.at[r,df_freq.columns[c]] = math.log2(df_freq.at[r, df_freq.columns[c]])

  else:
    df_freq = pd.DataFrame()
  
  return df_freq

## Calc Ranking

In [0]:
def calc_rank(df_freq, tipo_busca):
  if(df_freq.index.size > 0):
    dv = []
    qv = []

    if tipo_busca == 'livre':
      for q in df_freq['Terms'].values:
        qv.append(1)
      df_freq.at[len(df_freq.index),'Terms'] = 'cosine'
      df_freq.index = df_freq['Terms']
    else:
      for q in df_freq['Field Text'].values:
        qv.append(1)
      df_freq.at[len(df_freq.index),'Field Text'] = 'cosine'
      df_freq.index = df_freq['Field Text']

    df_freq = df_freq.iloc[:, 1:]

    for c in range(1, len(df_freq.columns)):
      for r in range(len(df_freq.index[:-1])):
        dv.append(df_freq.at[df_freq.index[r],df_freq.columns[c]])
      df_freq.at[df_freq.index[len(df_freq.index)-1], df_freq.columns[c]] = (np.dot(dv, qv) / (np.linalg.norm(dv) * np.linalg.norm(qv)))
      dv = []

    df_rank = df_freq.sort_values(by='cosine', axis=1, ascending=False)
    df_rank = df_rank.drop(['Freq'], axis=1)
    ranking = df_rank.columns.tolist()
    ranking = ranking[:10]
  else:
    ranking = 'Busca não encontrada!'
    
  return ranking

In [0]:
def ranking_dictionary(r):
  if(r == 'Busca não encontrada!'):
    rd_list = 'Busca não encontrada!'
  else:
    rd_list = list()
    for i in r:
      path = df_arquivos.at[int(i), '0']
      row, = df_ed[df_ed['Documento'] == path].index
      titulo = df_ed.at[row, 'Título']
      if type(titulo) != str:
        titulo = 'Unknown'
      autor = df_ed.at[row, 'Autor']
      if type(autor) != str:
        autor = 'Unknown'
      print(autor)
      rd = {
          'index': i,
          'title': titulo.lower(),
          'author': autor.lower()
      }
      rd_list.append(rd)

  return rd_list

## Cálculo da Correlação de Spearman



In [0]:
def calc_spearman(r1, r2):
  squares = list()
  ranking1 = r1
  ranking2 = r2
  for i in range(len(ranking1)):
    for j in range(len(ranking2)):
      if ranking1[i] == ranking2[j]:
        sub = i - j
        square = math.pow(sub, 2)
      else:
        square = 0
      squares.append(square)

  d = ranking1 + ranking2
  d = list(dict.fromkeys(d))
  k = len(d)
  spearman_corr = 1 - ((6*sum(squares))/(k*(math.pow(k, 2)-1)))
  return spearman_corr

# Main

## Índice

In [263]:
df_indice = pd.read_csv('/content/drive/My Drive/indice_invertido_all_view_pages.csv')
df_indice.head(n=10)

,Terms,Freq,Posting,Compress Posting,Gamma Code
0,abaixo,38,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,abordagem,3,"[7, 64, 74]","[7, 57, 10]","[11011, 11111011001, 1110010]"
2,abrir,20,"[82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 9...","[82, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[1111110010010, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,abuso,8,"[0, 1, 3, 4, 5, 6, 7, 8]","[0, 1, 2, 1, 1, 1, 1, 1]","[0, 0, 100, 0, 0, 0, 0, 0]"
4,acabam,6,"[6, 49, 52, 55, 56, 57]","[6, 43, 3, 3, 1, 1]","[11010, 11111001011, 101, 101, 0, 0]"
5,acabamento,19,"[5, 9, 10, 11, 12, 13, 14, 15, 16, 49, 50, 51,...","[5, 4, 1, 1, 1, 1, 1, 1, 1, 33, 1, 1, 1, 1, 1,...","[11001, 11000, 0, 0, 0, 0, 0, 0, 0, 1111100000..."
6,acabou,8,"[89, 92, 95, 96, 98, 99, 100, 101]","[89, 3, 3, 1, 2, 1, 1, 1]","[1111110011001, 101, 101, 0, 100, 0, 0, 0]"
7,acaso,13,"[1, 13, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36...","[1, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 50]","[0, 1110100, 1110110, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,aceita,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 84]","[0, 1, 1, 1, 1, 1, 1, 1, 1, 76]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1111110001100]"
9,aceitamos,10,"[27, 28, 29, 30, 31, 32, 33, 34, 35, 36]","[27, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[111101011, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [264]:
df_arquivos = pd.read_csv('/content/drive/My Drive/urls_df.csv')
df_arquivos.head()

,0
0,/content/drive/My Drive/9periodo/Recuperação...
1,/content/drive/My Drive/9periodo/Recuperação...
2,/content/drive/My Drive/9periodo/Recuperação...
3,/content/drive/My Drive/9periodo/Recuperação...
4,/content/drive/My Drive/9periodo/Recuperação...


In [265]:
df_campos = pd.read_csv('/content/drive/My Drive/field_texts.csv')
df_campos.head(n=10)

,word,Field Text,Freq,Posting,Compress Posting
0,0,1808.title,1,[0],[0]
1,1,1900.title,1,[4],[4]
2,0,1967.publishing,1,[29],[29]
3,1,1984.publishing,1,[32],[32]
4,2,1987.publishing,1,[34],[34]
5,3,1991.publishing,1,[33],[33]
6,4,2006.publishing,1,[35],[35]
7,5,2019.publishing,1,[6],[6]
8,2,2019.title,2,"[11, 26]","[11, 15]"
9,0,8516079732.isbn,1,[51],[51]


## Busca livre

In [288]:
fq = 'vidas secas'
df_fs = free_search(fq,df_indice)
df_fs.head(n=10)

,Terms,Freq,Posting,Compress Posting,Gamma Code
0,vida,53,"[1, 2, 4, 9, 10, 12, 13, 14, 19, 21, 22, 23, 2...","[1, 1, 2, 5, 1, 2, 1, 1, 5, 2, 1, 1, 1, 2, 7, ...","[0, 0, 100, 11001, 0, 100, 0, 0, 11001, 100, 0..."
1,seca,19,"[26, 49, 52, 53, 55, 56, 57, 58, 84, 92, 93, 9...","[26, 23, 3, 1, 2, 1, 1, 1, 26, 8, 1, 1, 1, 1, ...","[111101010, 111100111, 101, 0, 100, 0, 0, 0, 1..."


In [289]:
dl_fs = docs_list_OR(df_fs)
print(dl_fs)

['1', '2', '4', '9', '10', '12', '13', '14', '19', '21', '22', '23', '24', '26', '33', '35', '36', '37', '46', '48', '50', '52', '53', '54', '55', '56', '57', '58', '59', '61', '65', '66', '67', '69', '70', '72', '73', '78', '82', '83', '85', '86', '88', '89', '90', '91', '92', '94', '96', '98', '99', '100', '101', '49', '84', '93', '95', '97']


### Usando a frequência (tf)

In [323]:
df_f_fs = calc_freq(dl_fs, df_fs, 'livre', handlingQuery(fq))
df_f_fs.head(n=10)

,Terms,Freq,1,2,4,9,10,12,13,14,19,21,22,23,24,26,33,35,36,37,46,48,50,52,53,54,55,56,57,58,59,61,65,66,67,69,70,72,73,78,82,83,85,86,88,89,90,91,92,94,96,98,99,100,101,49,84,93,95,97
0,vida,53.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.807355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,seca,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.807355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [324]:
r_f_fs = calc_rank(df_f_fs, 'livre')
print(r_f_fs)

['26', '10', '1', '2', '4', '9', '12', '13', '14', '19']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


In [334]:
rd_f_fs = ranking_dictionary(r_f_fs)
print(rd_f_fs)

  Graciliano Ramos
Isaacson,Walter, Isaacson,Walter
Monteiro Lobato
John Van de Ruit
Sigmund Freud
Carroll,Lewis
King,Stephen
Unknown
Adams,Douglas
  Paulo Rezzutti
[{'index': '26', 'title': 'vidas secas - 140ªed.(2019)', 'author': '  graciliano ramos'}, {'index': '10', 'title': 'steve jobs - a biografia', 'author': 'isaacson,walter, isaacson,walter'}, {'index': '1', 'title': 'box - as melhores aventuras do sítio do picapau amarelo', 'author': 'monteiro lobato'}, {'index': '2', 'title': 'cotoco. o diário de um garoto de 13 anos', 'author': 'john van de ruit'}, {'index': '4', 'title': 'freud (1900) a interpretação dos sonhos: obras completas volume 4', 'author': 'sigmund freud'}, {'index': '9', 'title': 'alice no país das maravilhas (classic edition)', 'author': 'carroll,lewis'}, {'index': '12', 'title': 'it - a coisa', 'author': 'king,stephen'}, {'index': '13', 'title': 'como fazer amigos e influenciar pessoas', 'author': 'unknown'}, {'index': '14', 'title': 'o guia do mochileiro das g

### Usando tf-idf

In [335]:
df_tfidf_fs = calc_freq(dl_fs, df_fs, 'livre', handlingQuery(fq))
df_tfidf_fs = calc_tfidf(df_tfidf_fs)
df_tfidf_fs.head(n=10)

,Terms,Freq,1,2,4,9,10,12,13,14,19,21,22,23,24,26,33,35,36,37,46,48,50,52,53,54,55,56,57,58,59,61,65,66,67,69,70,72,73,78,82,83,85,86,88,89,90,91,92,94,96,98,99,100,101,49,84,93,95,97
0,vida,0.944505,0.0,0.0,0.0,0.0,0.944505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.651560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,seca,2.424498,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.806426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [336]:
r_tfidf_fs = calc_rank(df_tfidf_fs, 'livre')
print(r_tfidf_fs)

['26', '10', '1', '2', '4', '9', '12', '13', '14', '19']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


In [337]:
rd_tfidf_fs = ranking_dictionary(r_tfidf_fs)
print(rd_tfidf_fs)

  Graciliano Ramos
Isaacson,Walter, Isaacson,Walter
Monteiro Lobato
John Van de Ruit
Sigmund Freud
Carroll,Lewis
King,Stephen
Unknown
Adams,Douglas
  Paulo Rezzutti
[{'index': '26', 'title': 'vidas secas - 140ªed.(2019)', 'author': '  graciliano ramos'}, {'index': '10', 'title': 'steve jobs - a biografia', 'author': 'isaacson,walter, isaacson,walter'}, {'index': '1', 'title': 'box - as melhores aventuras do sítio do picapau amarelo', 'author': 'monteiro lobato'}, {'index': '2', 'title': 'cotoco. o diário de um garoto de 13 anos', 'author': 'john van de ruit'}, {'index': '4', 'title': 'freud (1900) a interpretação dos sonhos: obras completas volume 4', 'author': 'sigmund freud'}, {'index': '9', 'title': 'alice no país das maravilhas (classic edition)', 'author': 'carroll,lewis'}, {'index': '12', 'title': 'it - a coisa', 'author': 'king,stephen'}, {'index': '13', 'title': 'como fazer amigos e influenciar pessoas', 'author': 'unknown'}, {'index': '14', 'title': 'o guia do mochileiro das g

## Busca estruturada

In [338]:
sq = {
    'title': 'sítio do picapau amarelo',
    'author': 'monteiro lobato'
}

df_ss = structured_search(sq, df_campos)
df_ss.head(n=10)

,word,Field Text,Freq,Posting,Compress Posting
0,306,sítio.title,1,[1],[1]
1,58,capa.title,1,[45],[45]
2,253,picapau.title,1,[1],[1]
3,13,amarelo.title,1,[1],[1]
4,147,monteiro.author,1,[1],[1]
5,121,lobato.author,1,[1],[1]


In [339]:
dl_ss = docs_list_OR(df_ss)
print(dl_ss)

['1', '45']


### Usando a frequência (tf)

In [340]:
df_f_ss = calc_freq(dl_ss, df_ss, 'estruturada', getValues(sq))
df_f_ss.head(n=10)

,Field Text,Freq,1,45
0,sítio.title,1.0,0.0,0.0
1,capa.title,1.0,0.0,0.0
2,picapau.title,1.0,0.0,0.0
3,amarelo.title,1.0,0.0,0.0
4,monteiro.author,1.0,0.0,0.0
5,lobato.author,1.0,0.0,0.0


In [341]:
r_f_ss = calc_rank(df_f_ss, 'estruturada')
print(r_f_ss)

['1', '45']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


In [342]:
rd_f_ss = ranking_dictionary(r_f_ss)
print(rd_f_ss)

Monteiro Lobato
Johan Van Lengen
[{'index': '1', 'title': 'box - as melhores aventuras do sítio do picapau amarelo', 'author': 'monteiro lobato'}, {'index': '45', 'title': 'manual do arquiteto descalço - capa dura', 'author': 'johan van lengen'}]


### Usando tf-idf

In [346]:
df_tfidf_ss = calc_freq(dl_ss, df_ss, 'estruturada', getValues(sq))
df_tfidf_ss = calc_tfidf(df_tfidf_ss)
df_tfidf_ss.head(n=10)

,Field Text,Freq,1,45
0,sítio.title,6.672425,0.0,0.0
1,capa.title,6.672425,0.0,0.0
2,picapau.title,6.672425,0.0,0.0
3,amarelo.title,6.672425,0.0,0.0
4,monteiro.author,6.672425,0.0,0.0
5,lobato.author,6.672425,0.0,0.0


In [347]:
r_tfidf_ss = calc_rank(df_tfidf_ss, 'estruturada')
print(r_tfidf_ss)

['1', '45']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


In [348]:
rd_tfidf_ss = ranking_dictionary(r_tfidf_ss)
print(rd_tfidf_ss)

Monteiro Lobato
Johan Van Lengen
[{'index': '1', 'title': 'box - as melhores aventuras do sítio do picapau amarelo', 'author': 'monteiro lobato'}, {'index': '45', 'title': 'manual do arquiteto descalço - capa dura', 'author': 'johan van lengen'}]


## Correlação de consultas

In [0]:
#queries

query1 = 'diário de um garoto'
query2 = 'vidas secas'
query3 = 'universo'
query4 = 'redes de computadores'
query5 = 'stephen king'

In [354]:
#computando query 1
df_q1 = free_search(query1,df_indice)
dl_q1 = docs_list_OR(df_q1)

#calculo do ranking usando tf para query 1
df_tf_q1 = calc_freq(dl_q1, df_q1, 'livre', handlingQuery(query1))
r_tf_q1 = calc_rank(df_tf_q1, 'livre')
print('Ranking query 1 usando tf: ', r_tf_q1) 

#calculo do ranking usando tf-idf para query1
df_tfidf_q1 = calc_freq(dl_q1, df_q1, 'livre', handlingQuery(query1))
df_tfidf_q1 = calc_tfidf(df_tfidf_q1)
r_tfidf_q1 = calc_rank(df_tfidf_q1, 'livre')
print('Ranking query 1 usando tf-idf: ', r_tfidf_q1)

cp1 = calc_spearman(r_tf_q1, r_tfidf_q1)
print('Correlação entre rankings por tf e tf-idf: ', cp1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Ranking query 1 usando tf:  ['2', '9', '69', '27', '66', '92', '1', '10', '11', '12']
Ranking query 1 usando tf-idf:  ['2', '9', '27', '66', '69', '92', '1', '10', '11', '12']
Correlação entre rankings por tf e tf-idf:  0.9636363636363636


In [356]:
#computando query 2
df_q2 = free_search(query2,df_indice)
dl_q2 = docs_list_OR(df_q2)

#calculo do ranking usando tf para query 2
df_tf_q2 = calc_freq(dl_q2, df_q2, 'livre', handlingQuery(query2))
r_tf_q2 = calc_rank(df_tf_q2, 'livre')
print('Ranking query 2 usando tf: ', r_tf_q2)

#calculo do ranking usando tf-idf para query 2
df_tfidf_q2 = calc_freq(dl_q2, df_q2, 'livre', handlingQuery(query2))
df_tfidf_q2 = calc_tfidf(df_tfidf_q2)
r_tfidf_q2 = calc_rank(df_tfidf_q2, 'livre')
print('Ranking query 2 usando tf-idf: ', r_tfidf_q2)

cp2 = calc_spearman(r_tf_q2, r_tfidf_q2)
print('Correlação entre rankings por tf e tf-idf: ', cp2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Ranking query 2 usando tf:  ['26', '10', '1', '2', '4', '9', '12', '13', '14', '19']
Ranking query 2 usando tf-idf:  ['26', '10', '1', '2', '4', '9', '12', '13', '14', '19']
Correlação entre rankings por tf e tf-idf:  1.0


In [357]:
#computando query 3
df_q3 = free_search(query3,df_indice)
dl_q3 = docs_list_OR(df_q3)

#calculo do ranking usando tf para query 3
df_tf_q3 = calc_freq(dl_q3, df_q3, 'livre', handlingQuery(query3))
r_tf_q3 = calc_rank(df_tf_q3, 'livre')
print('Ranking query 3 usando tf: ', r_tf_q3)

#calculo do ranking usando tf-idf para query 3
df_tfidf_q3 = calc_freq(dl_q3, df_q3, 'livre', handlingQuery(query3))
df_tfidf_q3 = calc_tfidf(df_tfidf_q3)
r_tfidf_q3 = calc_rank(df_tfidf_q3, 'livre')
print('Ranking query 3 usando tf-idf: ', r_tfidf_q3)

cp3 = calc_spearman(r_tf_q3, r_tfidf_q3)
print('Correlação entre rankings por tf e tf-idf: ', cp3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Ranking query 3 usando tf:  ['3', '10', '14', '16', '17', '34', '78', '92', '94', '96']
Ranking query 3 usando tf-idf:  ['3', '10', '14', '16', '17', '34', '78', '92', '94', '96']
Correlação entre rankings por tf e tf-idf:  1.0


In [358]:
#computando query 4
df_q4 = free_search(query4,df_indice)
dl_q4 = docs_list_OR(df_q4)

#calculo do ranking usando tf para query 4
df_tf_q4 = calc_freq(dl_q4, df_q4, 'livre', handlingQuery(query4))
r_tf_q4 = calc_rank(df_tf_q4, 'livre')
print('Ranking query 4 usando tf: ', r_tf_q4)

#calculo do ranking usando tf-idf para query 4
df_tfidf_q4 = calc_freq(dl_q4, df_q4, 'livre', handlingQuery(query4))
df_tfidf_q4 = calc_tfidf(df_tfidf_q4)
r_tfidf_q4 = calc_rank(df_tfidf_q4, 'livre')
print('Ranking query 4 usando tf-idf: ', r_tfidf_q4)

cp4 = calc_spearman(r_tf_q4, r_tfidf_q4)
print('Correlação entre rankings por tf e tf-idf: ', cp4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Ranking query 4 usando tf:  ['7', '82', '83', '84', '85', '86', '87', '88', '89', '90']
Ranking query 4 usando tf-idf:  ['7', '82', '83', '84', '85', '86', '87', '88', '89', '90']
Correlação entre rankings por tf e tf-idf:  1.0


In [359]:
#computando query 5
df_q5 = free_search(query5,df_indice)
dl_q5 = docs_list_OR(df_q5)

#calculo do ranking usando tf para query 5
df_tf_q5 = calc_freq(dl_q5, df_q5, 'livre', handlingQuery(query5))
r_tf_q5 = calc_rank(df_tf_q5, 'livre')
print('Ranking query 5 usando tf: ', r_tf_q5)

#calculo do ranking usando tf-idf para query 5
df_tfidf_q5 = calc_freq(dl_q5, df_q5, 'livre', handlingQuery(query5))
df_tfidf_q5 = calc_tfidf(df_tfidf_q5)
r_tfidf_q5 = calc_rank(df_tfidf_q5, 'livre')
print('Ranking query 5 usando tf-idf: ', r_tfidf_q5)

#calculo da correlação entre rankings
cp5 = calc_spearman(r_tf_q5, r_tfidf_q5)
print('Correlação entre rankings por tf e tf-idf: ', cp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Ranking query 5 usando tf:  ['83', '92', '93', '94', '95', '97', '98', '99', '12', '100']
Ranking query 5 usando tf-idf:  ['83', '92', '93', '94', '95', '97', '98', '99', '12', '100']
Correlação entre rankings por tf e tf-idf:  1.0
